In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.models import model_from_json
json_file = open('/content/drive/My Drive/LITS/model_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/LITS/model_weights.h5")
print("Loaded model from disk")

Using TensorFlow backend.


Loaded model from disk


In [0]:
import nibabel as nib
img_path='/content/drive/My Drive/LITS/Dataset/volume-23.nii'
img_ex = nib.load(img_path).get_data()
#mask_ex = nib.load(mask_path[25]).get_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
patch_ratio = []

for i in range(16 + 1):
  patch_ratio.append(32 * i)

In [0]:
def slice_to_patch(slice, patch_ratio):
  
  slice[slice == 1] = 0
  slice[slice == 2] = 1
  
  patch_list = []
  
  for x_bin in range(2, len(patch_ratio)):
    for y_bin in range(2, len(patch_ratio)):
      patch = slice[patch_ratio[x_bin-2] : patch_ratio[x_bin], patch_ratio[y_bin - 2] : patch_ratio[y_bin]]
      patch = patch.reshape(patch.shape + (1,))
      patch_list.append(patch)
  
  return np.array(patch_list)

In [0]:
def patch_to_slice(patch, patch_ratio, input_shape, conf_threshold):
  
  slice = np.zeros((512, 512, 1))
  row_idx = 0
  col_idx = 0
  
  for i in range(len(patch)):
    
    slice[patch_ratio[row_idx]:patch_ratio[row_idx + 2], patch_ratio[col_idx]:patch_ratio[col_idx + 2]][patch[i] > conf_threshold] = 1
    
    col_idx += 1
    
    if i != 0 and (i+1) % 15 == 0:
      row_idx += 1
      col_idx = 0
  
  return slice

In [0]:
import numpy as np
import matplotlib.pyplot as plt
input_shape = [64, 64, 1]

for i in range(img_ex.shape[2]):  
        
      patch_ex = slice_to_patch(img_ex[:, :, i], patch_ratio)
      prediction = loaded_model.predict(patch_ex)
      prediction_mask = patch_to_slice(prediction, patch_ratio, input_shape, conf_threshold = 0.97)
      
      fig, (ax1,ax3) = plt.subplots(1, 2, figsize = ((15, 15)))
      
      ax1.imshow(np.rot90(img_ex[:, :, i], 3), cmap = 'bone')
      ax1.set_title("Image", fontsize = "x-large")
      ax1.grid(False)
  
      ax3.imshow(np.rot90(prediction_mask.reshape((512, 512)), 3), cmap = 'bone')
      ax3.set_title("Mask (Pred)", fontsize = "x-large")
      ax3.grid(False)
      plt.show()